# Optical character recognition (OCR) 
Project 4 Group 3


## Error Detection

First of all, we need to detect errors, or incorrectly processed words. Here we extract features according to the paper and use SVM for garbage detection.

In [15]:
import os
import glob
import re
from collections import Counter
import pandas as pd
import collections
import numpy as np
from sklearn.svm import SVC
import itertools
#import itertools
#nltk.download('punkt')

In [16]:
##### read ground_truth
ground_dir = glob.glob(os.path.join(os.getcwd(),'../data/ground_truth','*.txt'))
ground_tokens = []
for gd in ground_dir:
    with open(gd) as ground_file:
        ground_raw = ground_file.read()
        ground_t = ground_raw.split()   
        ground_tokens += ground_t
print(len(ground_tokens))
print(ground_tokens[:20])

294841
['communications', 'network.', 'Member', 'companies', 'are', 'strongly', 'encouraged', 'to', 'provide', 'this', 'needed', 'support.', 'The', 'state', 'advocacy', 'program*', 'including', 'the', 'new', 'CMA/LINC']


In [17]:
tess_dir = glob.glob(os.path.join(os.getcwd(),'../data/tesseract','*.txt'))
#ground_dir = glob.glob(os.path.join(os.getcwd(),'data/ground_truth','*.txt'))
file_name_gd = []
file_name_td = []
for gd, td in zip(ground_dir, tess_dir):
        with open(gd, encoding="utf8") as ground_file:    #, encoding="utf8"
            with open(td, encoding="utf8") as tess_file:                
                ground_r = list(ground_file.readlines()) 
                tess_r = list(tess_file.readlines())
                if len(tess_r) == len(ground_r):
                    file_name_td.append(td)
                    file_name_gd.append(gd)

In [18]:
# make sure all the lines have the same size.
ground_tokens=[]
tess_tokens=[]
for gd, td in zip(file_name_gd, file_name_td):
        with open(gd) as file1:
            with open(td) as file2:
                for line1,line2 in zip(file1,file2):
                    if len(line1)==len(line2):
                        for word1,word2 in zip(line1.split(),line2.split()):
                            ground_tokens.append(word1)
                            tess_tokens.append(word2)

In [19]:
print(tess_tokens[:30])
print(ground_tokens[:30])

['communlcatlons', 'network.', 'Member', 'companles', 'are', 'strongly', 'encouraged', 'to', 'provlde', 'thls', 'needed', 'support.', 'The', 'state', 'advocacy', 'program"', '1nclud1ng', 'the', 'new', 'CMA/LINC', 'computer', 'network.', 'will', 'be', 'heavlly', '1nvolved', '1n', '1995', '1n', 'the']
['communications', 'network.', 'Member', 'companies', 'are', 'strongly', 'encouraged', 'to', 'provide', 'this', 'needed', 'support.', 'The', 'state', 'advocacy', 'program*', 'including', 'the', 'new', 'CMA/LINC', 'computer', 'network,', 'will', 'be', 'heavily', 'involved', 'in', '1986', 'in', 'the']


In [20]:
print(len(tess_tokens))
print(len(ground_tokens))

152143
152143


In [21]:
y = []
for gt, tt in zip(ground_tokens, tess_tokens):
        if gt == tt:
            y.append(0)   # 0 indicates correct
        else:
            y.append(1)   # 1 indicates error

In [22]:
X = pd.DataFrame(tess_tokens)   
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
ground_train = [ground_tokens[i] for i in X_train.index.tolist()]
ground_test = [ground_tokens[i] for i in X_test.index.tolist()]
X_train = X_train[0].tolist()
X_test = X_test[0].tolist()

In [23]:
print(X_train[:30])
print(ground_train[:30])
print(y_train[:30])

['The', 'Attachment', 'the', '1n', 'force', 'a', 'complex.', 'proposed', 'incoming', 'NO', 'trllllon', 'August,', 'New', 'Durlng', 'over', 'focus', 'and', 'credlt', 'Manufacturers', 'and', 'Agreement', 'as', 'trenches,', 'for', 'on', 'Page', 'testlfled', 'that', '15', 'admlnlstratlon']
['The', 'Attachment', 'the', 'in', 'force', 'a', 'complex.', 'proposed', 'president', 'NO', 'trillion', 'August,', 'New', 'During', 'over', 'focus', 'and', 'credit', 'Manufacturers', 'and', 'Agreement', 'as', 'trenches,', 'for', 'on', 'Page', 'testified', 'that', 'is', 'administration']
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]


In [24]:
# define character type
vowels = list('aeiou')
consonants = list("bcdfghjklmnpqrstvwxyz")
digits = list('0123456789')

def count_up(word):
    u = [x for x in word if x.isupper()]
    return len(u)

def count_low(word):
    l = [x for x in word if x.islower()]
    return len(l)

# count the number of consecutive of the same symbol (if greater than 3 => garbadge)
def count_cons_occur(word):
    max_occ=0
    previous=None
    temp=1
    for i in word:
        if i==previous:
            temp=temp+1
        else:
            max_occ=max(max_occ,temp)
            temp=1
        previous=i
    return(max_occ)

#  count the number of consecutive of the consonants(if greater than 6 => garbadge)
def count_cons_occur_consonants(word):
    max_occ=0
    temp=0
    for i in word:
        if i in consonants:
            temp=temp+1
        else:
            max_occ=max(max_occ,temp)
            temp=0
    return(max_occ)

# get the subset of the word: delete the 1st and last word 
def subset_word(word):
    return word[1:len(word) - 1]

#  do the division in case of  prevent from divisor = 0        
def div(x,y):
    if y != 0:
        return x/y
    return 0

In [25]:
# get the bigram frequency for the particular word (return the array of each bf_i)
def bigram_freq(word, bi_dict):
    word = word.lower()
    bf = []
    for i in range(len(word)-1):
        key = word[i:i+2]
        bf.append(bi_dict[key])
    return(bf)

# Form the list of letter bigrams from our ground truth dataset -- bi_dict
# First we normalized all tokens to the lowercase 
lower_case_words = []
for word in ground_tokens:
    lower_word = word.lower()
    lower_case_words.append(lower_word)

# Secondly, we form the LB from our ground_truth dataset 
# create a dictionary-like collection: key: bigrams value: frequency 
LB = collections.defaultdict(int)
for word in lower_case_words:
    for i in range(len(word)-1):
        key = word[i:i+2]
        LB[key] +=1

In [26]:
def feature_matrix(data):
    ##### Feature extracting
    
    # feature 1 : length of the each words
    length = []

    # feature 2: number of each vowels and consonants; and the corresponding ratio
    vowels_count = []
    consonants_count = []
    ratio_v_l = []
    ratio_c_l = []
    ratio_v_c = []

    # feature 3: number of special symbols: not vowels, not consonants and not digits 
    nonalpha = []

    # feature 4: number of digits and the ratio bewteen number of digit and length of the words 
    digits = []
    ratio_d_l = []

    # feature 5: number of lowercase chars, uppercase chars and corresponding ratio 
    lower_case = []
    upper_case = []
    ratio_low_l = []
    ratio_up_l = []
    
    # feature 6: number of consecutive occurances of the same symbol and the ratio between it and the length
    consecutive_occur = []
    ratio_sym_l = []
    
    # feature 7: number of all alpha(words) and numbers in the words and the ratio between it and the length
    # total number of vowels, consonants and digits
    ratio_la = [] 
    
    # feature 8: number of consecutive consonants in the word
    consonants_consecutive = []
    
    # feature 9: delete the 1st and the last symbol and then check the number of non alpha - number  character 
    infix_nonalpha = []

    # feature 10: bigram frequency 
    bigram = []
    
    # feature 11: number of most frequent symbols (including alpha,numbers, special symbols)
    most_freq = []
    
    # feature 12: number of non-alpha symbols: length(words) - length(alpha)
    l2 = []
    
    
        # feature 1:
    for word in data:
        L= len(word)
        length.append(L)

        # feature 2: vowels
        word_lower = word.lower()
        v_count = len([v for v in word if v in vowels])
        vowels_count.append(v_count)
        
        # feature 2: consonants
        c_count = len([c for c in word if c in consonants])
        consonants_count.append(c_count)
   
        ratio_v_l.append(v_count/L)
        ratio_c_l.append(c_count/L)
        ratio_vc = div(v_count,c_count)
        ratio_v_c.append(ratio_vc)

        # feature 3
        symbol_count = len([s for s in word if s not in vowels or consonants or digits])
        nonalpha.append(symbol_count)
     
        # feature 4
        d_count = len([d for d in word if d in digits])
        digits.append(d_count)
        ratio_d_l.append(d_count/L)
        
        # feature 5
        low_count = count_low(word)
        lower_case.append(low_count)
        upper_count = count_up(word)
        upper_case.append(upper_count)

        ratio_low_l.append(low_count/L)
        ratio_up_l.append(upper_count/L)

        # feature 6
        cons_occur_count = int(count_cons_occur(word))
        consecutive_occur.append(cons_occur_count)
        if cons_occur_count >= 3:
            ratio_sym_l.append(cons_occur_count/L)
        else:
            ratio_sym_l.append(0)

        # feature 7
        la = L-symbol_count
        if symbol_count > la:
            ratio_la.append(1)
        else:
            ratio_la.append(0)

        # feature 8
        consonance_consecu_count = int(count_cons_occur_consonants(word))
        if consonance_consecu_count >= 6:
            consonants_consecutive.append(1)
        else:
            consonants_consecutive.append(0)

        # feature 9 
        # delete the 1st and last symbol of the string: remaining(2 or more special symbol) = 1; otherwise, = 0
        word_removed = subset_word(word)
        remove_count = len([r for r in word_removed if r not in vowels or consonants or digits])
        if remove_count >=3:
            infix_nonalpha.append(1)
        else:
            infix_nonalpha.append(0)  

        # feature 10 bigram
        bf = bigram_freq(word, LB)
    
        lower_tess_tokens = []
        #  = word.lower()
        lower_tess_tokens.append(word_lower)
        
        # number of bigrams of the string 
        n = len(lower_tess_tokens)-1
        # bigr: (sum(bf_i)/10000) / n
        big = div((sum(bf)/10000),n)
        bigram.append(big)

        # feature 11 most frequent symbol
        i_count = Counter(word).most_common(1)[0][1]
        if i_count >=3:
            most_freq.append(i_count/L)   # i/l
        else:
            most_freq.append(0)

        # feature 12 Non-alphabetical symbols: nonalpha/total symbols
        l1 = len([v for v in word_lower if v in vowels]) + len([c for c in word_lower if c in consonants])
        l2_count = L - l1
        ratio_l2 = div(l2_count,l1)
        l2.append(ratio_l2)  
        
        # construct a feature dataframe for SVM
    df1 = pd.DataFrame({'length': length,
                        'vowels': vowels_count,
                        'consonants': consonants_count,
                        'quot v/l': ratio_v_l,
                        'quot c/l': ratio_c_l,
                        'quot v/c': ratio_v_c,
                        'nonalpha': nonalpha,
                        'digits': digits,
                        'quot d/l': ratio_d_l,
                        'lowers': lower_case,
                        'uppers': upper_case,
                        'quot low/l': ratio_low_l,
                        'quot up/l': ratio_up_l,
                        'cons occur': consecutive_occur,
                        'quot cons/l': ratio_sym_l,
                        'quot la': ratio_la,
                        'consonants_secut': consonants_consecutive,
                        'fix_nonalpha': infix_nonalpha,
                        'bigr': bigram,
                        'most_freq' : most_freq,
                        'l2': l2})
    return df1

In [28]:
X_feature_train = feature_matrix(X_train)
X_feature_test = feature_matrix(X_test)

In [32]:
X_feature_train.to_csv('../output/X_feature_train')
X_feature_test.to_csv('../output/X_feature_test')

In [29]:
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_feature_train, y_train)  

/Users/shengweihuang/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [33]:
#save model
import pickle
filename = '../output/SVM_model.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [34]:
#this chunk is for load model
import pickle
filename = '../output/SVM_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [35]:
loaded_model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [36]:
#to predict
y_pred = svclassifier.predict(X_feature_test)  
# y_pred = svm_model.predict(X_feature_test)  

In [41]:
df_output = pd.DataFrame({'tokens_tesseract':X_test,
                          'Predict_by_SVM': y_pred,'True word':ground_test})
print(df_output[:100])

   tokens_tesseract  Predict_by_SVM      True word
0         countrles               1      countries
1           number.               0        number.
2              slze               0             of
3        membershlp               1     membership
4               out               0            out
5            relent               0         relent
6       asslstance.               1    assistance.
7           strong,               1        strong,
8           requlre               0       industry
9         Congress.               1      Congress.
10        producers               1      producers
11              the               0            the
12          sildeS.               0        slides,
13               by               0              5
14              You               0            You
15               1n               1              a
16            would               0          would
17        1ndustry.               1      industry.
18             give            

In [39]:
##### evaluation
#confustion Matrix
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  

[[15747  2376]
 [ 3675  8631]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84     18123
           1       0.78      0.70      0.74     12306

   micro avg       0.80      0.80      0.80     30429
   macro avg       0.80      0.79      0.79     30429
weighted avg       0.80      0.80      0.80     30429

